Device tree overlay

In [ ]:
import os

class overlay (object):

    def __init__ (self, overlay:str):
        if not isinstance(overlay, str):
            raise TypeError("Bitstream name has to be a string.")

        if os.path.isfile("{}.dts".format(overlay)):
            self.overlay = overlay
        else:
            raise IOError('Device tree overlay {}.dts does not exist.'.format(overlay))

        os.system("dtc -I dts -O dtb -o {overlay}.dtbo -@ {overlay}.dts".format(overlay=overlay))
        os.system('cat /opt/redpitaya/fpga/{}/fpga.bit > /dev/xdevcfg'.format(overlay))
        os.system("mkdir /sys/kernel/config/device-tree/overlays/{}".format(overlay))
        os.system("cat {overlay}.dtbo > /sys/kernel/config/device-tree/overlays/{overlay}/dtbo".format(overlay=overlay))

    def __del__ (self):
        os.system("rmdir /sys/kernel/config/device-tree/overlays/{}".format(overlay))
        self.overlay = None

In [ ]:
o = overlay("mercury")

In [ ]:
import os
import mmap
import numpy as np

class hwid ():

    regset_dtype = np.dtype([
        ('hwid' , 'uint32'),
        ('rsv0' , 'uint32'),  # reserved
        ('efuse', 'uint32'),
        ('rsv1' , 'uint32'),  # reserved
        ('dna'  , 'uint32', 2),
        ('rsv3' , 'uint32', 2),  # reserved
        ('gith' , 'uint32', 5)
    ])

    def __init__ (self, uio:str = '/dev/uio/hwid'):
        # TODO add exclusive lock
        try:
            self.uio_dev = os.open(uio, os.O_RDWR | os.O_SYNC)
        except OSError as e:
            raise IOError(e.errno, "Opening {}: {}".format(uio, e.strerror))

        try:
            self.uio_mem = mmap.mmap(
                fileno=self.uio_dev, length=mmap.PAGESIZE, offset=0x0,
                flags=mmap.MAP_SHARED, prot=(mmap.PROT_READ | mmap.PROT_WRITE))
        except OSError as e:
            raise IOError(e.errno, "Mapping {}: {}".format(uio, e.strerror))

        regset_array = np.recarray(1, self.regset_dtype, buf=self.uio_mem)
        self.regset = regset_array[0]
        
    def __del__ (self):
        self.uio_mem.close()
        try:
            os.close(self.uio_dev)
        except OSError as e:
            raise IOError(e.errno, "Closing {}: {}".format(uio, e.strerror))

In [ ]:
regid = hwid();

In [ ]:
del(regid)

In [ ]:
regid.regset

In [ ]:
class asg ():

    regset_dtype = np.dtype([
        # control/status
        ('ctl_sts', 'uint32'),
        # trigger configuration
        ('cfg_trg', 'uint32'),  # trigger mask
        ('rsv0'   , 'uint32', 2),  # reserved
        # buffer configuration
        ('cfg_siz', 'uint32'),  # size
        ('cfg_off', 'uint32'),  # offset
        ('cfg_stp', 'uint32'),  # step
        ('rsv1'   , 'uint32'),  # reserved
        # burst mode
        ('cfg_bmd', 'uint32'),  # mode [1:0] = [inf, ben]
        ('cfg_bdl', 'uint32'),  # data length
        ('cfg_bln', 'uint32'),  # length (data+pause)
        ('cfg_bnm', 'uint32'),  # number of bursts pulses
        # burst status
        ('sts_bln', 'uint32'),  # length (current position inside burst length)
        ('sts_bnm', 'uint32'),  # number (current burst counter)
        # linear transformation
        ('cfg_mul',  'int32'),  # multiplier (amplitude)
        ('cfg_sum',  'int32')   # addedr (offset)
    ])

    def __init__ (self, index:int, uio:str = '/dev/uio/asg'):
        uio = uio+str(index)
        # TODO add exclusive lock
        try:
            self.uio_dev = os.open(uio, os.O_RDWR | os.O_SYNC)
        except OSError as e:
            raise IOError(e.errno, "Opening {}: {}".format(uio, e.strerror))

        try:
            self.uio_mem = mmap.mmap(
                fileno=self.uio_dev, length=mmap.PAGESIZE, offset=0x0,
                flags=mmap.MAP_SHARED, prot=(mmap.PROT_READ | mmap.PROT_WRITE))
        except OSError as e:
            raise IOError(e.errno, "Mapping {}: {}".format(uio, e.strerror))

        regset_array = np.recarray(1, self.regset_dtype, buf=self.uio_mem)
        self.regset = regset_array[0]
        
    def __del__ (self):
        self.uio_mem.close()
        try:
            os.close(self.uio_dev)
        except OSError as e:
            raise IOError(e.errno, "Closing {}: {}".format(uio, e.strerror))